In [7]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head() 

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,49.57,99,100,2.21,moderate rain
1,1,Cherskiy,RU,68.7500,161.3000,55.98,39,2,5.01,clear sky
2,2,Castro,BR,-24.7911,-50.0119,49.15,78,0,2.42,clear sky
3,3,Beloha,MG,-25.1667,45.0500,66.36,74,0,4.88,clear sky
4,4,Saskylakh,RU,71.9167,114.0833,50.81,54,98,8.01,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Price,US,39.5994,-110.8107,91.11,6,1,3.44,clear sky
8,8,Kavieng,PG,-2.5744,150.7967,83.77,69,2,10.42,clear sky
10,10,Atuona,PF,-9.8000,-139.0333,79.11,76,48,17.05,scattered clouds
12,12,Sur,OM,22.5667,59.5289,87.78,62,0,7.94,clear sky
13,13,Rikitea,PF,-23.1203,-134.9692,76.33,74,99,10.69,overcast clouds
14,14,Avera,US,33.1940,-82.5271,80.74,64,63,2.73,broken clouds
17,17,Damghan,IR,36.1683,54.3480,78.66,37,0,3.60,clear sky
31,31,Faanui,PF,-16.4833,-151.7500,80.29,77,37,15.21,scattered clouds
32,32,Dhidhdhoo,MV,6.8833,73.1000,81.82,77,98,15.99,overcast clouds
33,33,Awbari,LY,26.5921,12.7805,79.32,25,0,3.69,clear sky


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

#There aren't any empty rows to drop in my DataFrame.


In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Price,US,91.11,clear sky,39.5994,-110.8107,
8,Kavieng,PG,83.77,clear sky,-2.5744,150.7967,
10,Atuona,PF,79.11,scattered clouds,-9.8000,-139.0333,
12,Sur,OM,87.78,clear sky,22.5667,59.5289,
13,Rikitea,PF,76.33,overcast clouds,-23.1203,-134.9692,
14,Avera,US,80.74,broken clouds,33.1940,-82.5271,
17,Damghan,IR,78.66,clear sky,36.1683,54.3480,
31,Faanui,PF,80.29,scattered clouds,-16.4833,-151.7500,
32,Dhidhdhoo,MV,81.82,overcast clouds,6.8833,73.1000,
33,Awbari,LY,79.32,clear sky,26.5921,12.7805,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

#There arent empty hotel names in my DataFrame


City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [23]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/Vacation_Search.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure

